# Intel Image Classification

This notebook will show how to do image classification with six different classes. The model will be constructed useing transfered learning implemented with the keras imagenet as the base model. The image data is from kaggle and can be found [here](https://www.kaggle.com/puneet6060/intel-image-classification).

In [1]:
# set up egpu using plaidml
import os

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [25]:
# import keras layers , activation funtions and optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras import optimizers

# dimensions of our images.
img_width, img_height = 300, 300

# set directory
train_data_dir = 'intel-image-classification/train'
validation_data_dir = 'intel-image-classification/test'

# steps per epoch
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 5
batch_size = 16

In [22]:
# imprt keras applications
from keras.layers import GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

# dowload imagenet weights without the final layer
base_model=MobileNet(weights='imagenet',include_top=False) 

# add dense layers so that that the model performs better on our data
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
x=Dense(128,activation='relu')(x)
preds=Dense(6,activation='softmax')(x) #final layer with softmax activation and six outputs

In [16]:
# import keras Model to connect the base model and our custom model outputs
from keras.models import Model

# specify model inputs and outputs
model=Model(inputs=base_model.input,outputs=preds)

In [17]:
# freeze the base model parameters
for layer in model.layers[:20]:
    layer.trainable=False
# make final 20 model layer traianble 
for layer in model.layers[20:]:
    layer.trainable=True

In [18]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
rescale=1. / 255,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True)

In [19]:
# this is the augmentation configuration we will use for testing
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode='binary')

# this is the augmentation configuration we will use for testing
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode='binary')

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [23]:
# define optimizer
opt = optimizers.Nadam(lr=.0001)
# compile model
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [26]:
# fit model to train and validation datasets
model.fit_generator(
    train_generator,
steps_per_epoch=nb_train_samples // batch_size,
epochs=epochs,
validation_data=validation_generator,
validation_steps=nb_validation_samples // batch_size)
model.save_weights('fifth_try.h5')

Epoch 1/5
125/125 [==============================] - 69s 551ms/step - loss: 0.2619 - acc: 0.9065 - val_loss: 0.3436 - val_acc: 0.8875
Epoch 2/5
125/125 [==============================] - 68s 548ms/step - loss: 0.2278 - acc: 0.9260 - val_loss: 0.3589 - val_acc: 0.8838
Epoch 3/5
125/125 [==============================] - 69s 553ms/step - loss: 0.2428 - acc: 0.9200 - val_loss: 0.3173 - val_acc: 0.9025
Epoch 4/5
125/125 [==============================] - 69s 553ms/step - loss: 0.2319 - acc: 0.9150 - val_loss: 0.3481 - val_acc: 0.8889
Epoch 5/5
125/125 [==============================] - 70s 557ms/step - loss: 0.2300 - acc: 0.9195 - val_loss: 0.3037 - val_acc: 0.8988
